In [3]:
import streamlit as st
import numpy as np
import pandas as pd
import plotly.express as px
import fastf1
import os
from fastf1 import plotting
from matplotlib import pyplot as plt
fastf1.Cache.enable_cache('fastf1_cache')  # optional but recommended
results_path = os.path.join(os.getcwd(), "results_cache")

In [6]:
def get_gps_of_year(year: int):
    gp_list = []
    event_schedule = fastf1.get_event_schedule(year)
    gp_list = list(event_schedule["EventName"].iloc[1:])
    gp_list
    return gp_list


def get_saison_results(year: int):
    gp_list = get_gps_of_year(year)
    results_saison = pd.DataFrame([], columns=["Year","GP","Driver","Team","Points"])

    for gp in gp_list:
        race = fastf1.get_session(year, gp, 'R')
        race.load()
        results = race.results
        for index, row in results.iterrows():
            results_saison.loc[len(results_saison)] = [year, gp_list.index(gp), row["FullName"], row["TeamName"], row["Points"]]

    results_saison.to_csv("results_cache/results" + str(year) + ".csv")
    return

In [11]:
for year in range(2000, 2019):
    get_saison_results(year)

2022-09-27 14:40:24.689 INFO    root: Loading data for Brazilian Grand Prix - Race [v2.3.0]
2022-09-27 14:40:24.848 WARNING root: Cannot load laps, telemetry, weather, and message data because the relevant API is not supported for this session.
2022-09-27 14:40:24.850 INFO    root: Finished loading data for 22 drivers: ['3', '11', '5', '6', '9', '10', '19', '18', '23', '21', '2', '8', '20', '1', '4', '7', '22', '14', '15', '12', '16', '17']
2022-09-27 14:40:24.968 INFO    root: Loading data for San Marino Grand Prix - Race [v2.3.0]
2022-09-27 14:40:25.140 WARNING root: Cannot load laps, telemetry, weather, and message data because the relevant API is not supported for this session.
2022-09-27 14:40:25.141 INFO    root: Finished loading data for 22 drivers: ['3', '1', '2', '4', '22', '17', '7', '16', '12', '8', '11', '23', '21', '19', '6', '18', '9', '14', '15', '10', '20', '5']
2022-09-27 14:40:25.329 INFO    root: Loading data for British Grand Prix - Race [v2.3.0]
2022-09-27 14:40:25

In [16]:
results_2020 = pd.read_csv("results_cache/results2020.csv")
results_2020.head()

,Unnamed: 0,Year,GP,Driver,Team,Points
0,0,2020,0,Lewis Hamilton,Mercedes,26.0
1,1,2020,0,Valtteri Bottas,Mercedes,18.0
2,2,2020,0,Max Verstappen,Red Bull Racing,15.0
3,3,2020,0,Charles Leclerc,Ferrari,12.0
4,4,2020,0,Pierre Gasly,AlphaTauri,10.0


In [127]:
gp_list = list(results_2021["GP"].unique())
for index, row in results_2021.iterrows():
    results_2021.iloc[index, 1] = gp_list.index(row["GP"])

In [135]:
results_2021.to_csv("results2021.csv")

In [129]:
def accumulate_points_per_season(season_df: pd.DataFrame):
    
    # 1. Driver
    season_df1 = season_df.sort_values(["Driver", "GP"])
    points = np.zeros([len(season_df), 1])
    prev_driver = ""

    for index, row in season_df1.iterrows():
        if prev_driver != row["Driver"]:
            total_points = row["Points"]
        else:
            total_points += row["Points"]
        points[index] = total_points
        prev_driver = row["Driver"]
    
    season_df["Acc Driver Points"] = points

    # 2. Team
    season_df2 = season_df.sort_values(["Team", "GP"])
    team_points = np.zeros([len(season_df), 1])
    prev_team = ""

    for index, row in season_df2.iterrows():
        if prev_team != row["Team"]:
            total_points = row["Points"]
        else:
            total_points += row["Points"]
        team_points[index] = total_points
        prev_team = row["Team"]
    
    season_df["Acc Team Points"] = team_points

    return season_df

In [141]:
accumulate_points_per_season(results_2021)

,Year,GP,Driver,Team,Points,Acc Driver Points,Acc Team Points
0,2021,0,Lewis Hamilton,Mercedes,25.0,25.0,25.0
1,2021,0,Max Verstappen,Red Bull Racing,18.0,18.0,18.0
2,2021,0,Valtteri Bottas,Mercedes,16.0,16.0,41.0
3,2021,0,Lando Norris,McLaren,12.0,12.0,12.0
4,2021,0,Sergio Perez,Red Bull Racing,10.0,10.0,28.0
...,...,...,...,...,...,...,...
434,2021,21,Sergio Perez,Red Bull Racing,0.0,190.0,578.5
435,2021,21,Nicholas Latifi,Williams,0.0,7.0,23.0
436,2021,21,Antonio Giovinazzi,Alfa Romeo Racing,0.0,3.0,13.0
437,2021,21,George Russell,Williams,0.0,16.0,23.0


In [131]:
def reward_relative_points(df: pd.DataFrame):

    # check if function already performed on DataFrame
    if "Relative Driver Points" in df.columns:
        print("function is already applied on this dataframe")
        return df_total

    first = True
    for year in df["Year"].unique():
        df_year = df[df["Year"]==year]

        points_in_season = df_year["Points"].sum()

        df_year["Relative Driver Points"] = df_year["Points"] / df_year["Points"].sum()
        df_year["Relative Acc Driver Points"] = df_year["Acc Driver Points"] / df_year["Points"].sum()
        df_year["Relative Acc Team Points"] = df_year["Acc Team Points"] / df_year["Points"].sum()

        if first == True:
            df_total = df_year
            first = False
        else: 
            df_total = pd.concat([df_total, df_year], axis=0)
        
    return df_total 

In [132]:
test2 = reward_relative_points(test)

In [163]:
def points_over_multiple_seasons(seasons_list_names: list):
    
    first = True
    total_GPs = 0
    
    for name_of_df in seasons_list_names:
        df_season = pd.read_csv(os.path.join(results_path, name_of_df))
        
        if first == True:
            df_season = accumulate_points_per_season(df_season)
            df_total = df_season
            first = False
        else:
            df_season = accumulate_points_per_season(df_season)
            df_season["GP"] = df_season["GP"] + total_GPs
            df_total = pd.concat([df_total, df_season], axis=0)

        total_GPs = df_season["GP"].max()

    df_rewarded = reward_relative_points(df_total)
    
    return df_rewarded

In [164]:
test3 = points_over_multiple_seasons(["results2020.csv", "results2021.csv", "results2022.csv"])

C:\Users\piuscarbon\AppData\Local\Temp\ipykernel_12348\849318866.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\piuscarbon\AppData\Local\Temp\ipykernel_12348\849318866.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\piuscarbon\AppData\Local\Temp\ipykernel_12348\849318866.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [167]:
fig = px.line(test3, x="GP", y="Relative Acc Team Points", color="Team", log_y=False)
fig.show()